# Permutation engine examples

This shows how to use the permutation engine to permute gates of a certain type towards the beginning or end of the circuit.
It relies on the implementation of permutation rules between different gates. Currently, only rotational gates + CNOTs + Pauli + S + H + T and TimeEvolution gates are supported. Arbitrary controlled gates are not supported and must be decomposed first. We provide a handy engine list that translates any circuit into the supported gate set and performes the permutation.

In [1]:
import projectq
import cmath
from projectq.backends import CommandPrinter, Simulator
from projectq.cengines import LocalOptimizer
from projectq.ops import *


from projqext.projectq.cengines import OpenSurgeryExporter
from projqext.projectq.cengines import PermutePi4Front, BasisRotation
from projqext.projectq.cengines import MultiqubitMeasurementCliffordEngine

#### First generate the main engine and the order of all the engines in the compiler stack.

In [2]:
engines = [CommandPrinter(),projectq.cengines.TagRemover(),PermutePi4Front()]
eng = projectq.MainEngine(backend=CommandPrinter(), engine_list=engines)

Now lets generate a very simple circuit

In [3]:
qubit = eng.allocate_qubit()
X | qubit
Y | qubit
X | qubit
T | qubit
T | qubit
eng.flush()

Allocate | Qureg[0]
X | Qureg[0]
Y | Qureg[0]
X | Qureg[0]
T | Qureg[0]
T | Qureg[0]
Allocate | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rx(3.14159265359) | Qureg[0]
Ry(3.14159265359) | Qureg[0]
Rx(3.14159265359) | Qureg[0]


Now lets take a look at a bit more complicated circuit that is still natively supported:

In [4]:
engines2 = [projectq.cengines.TagRemover(),PermutePi4Front(),MultiqubitMeasurementCliffordEngine(),CommandPrinter()]
eng2 = projectq.MainEngine(backend=OpenSurgeryExporter(output="output_instructions.txt"), engine_list=engines2, verbose=True)

qubit1 = eng2.allocate_qubit()
qubit2 = eng2.allocate_qubit()
qubit3 = eng2.allocate_qubit()
qubit4 = eng2.allocate_qubit()

T | qubit1
CNOT | (qubit3, qubit2)
Rx(-cmath.pi/2) | qubit4

CNOT | (qubit2, qubit1)
Rx(cmath.pi/2) | qubit3
Rz(cmath.pi/4) | qubit4

CNOT | (qubit4, qubit1)

Rz(cmath.pi/4) | qubit1
Rz(cmath.pi/2) | qubit2
Rz(cmath.pi/4) | qubit3
Rz(cmath.pi/2) | qubit4

Rx(-cmath.pi/2) | qubit1
Rx(cmath.pi/2) | qubit2
Rx(cmath.pi/2) | qubit3
Rx(cmath.pi/2) | qubit4

Measure | qubit1
Measure | qubit2
Measure | qubit3
Measure | qubit4

eng2.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
T | Qureg[0]
Ry(0.785398163397) | Qureg[3]
exp(0.3926990816985j * (1.0 Z0 Y1 Z2 Z3)) | Qureg[0, 3, 1-2]
exp(0.3926990816985j * (1.0 Y0 X1)) | Qureg[2, 1]
Parity Measurement Y0 Z1 Z2 Y3  | Qureg[0-3]
Parity Measurement X0 X1  | Qureg[0-1]
Parity Measurement Z0  | Qureg[2]
Parity Measurement X0 X1  | Qureg[0, 3]


This is actually the circuit shown in the paper: https://arxiv.org/abs/1808.02892

The last engine is an exporter that creates a file of instructions. The instruction set is then used by the surface code layouting tool for lattice surgery. Let's take a look at its content.

In [5]:
with open("output_instructions.txt") as fin:
    for line in fin:
        line = line.strip()
        print(line)

INIT 4
NEED A
MZZ A 0
MX A
S ANCILLA
MXX ANCILLA 0
H 3
S 3
NEED A
MZZ A 3
MX A
S ANCILLA
MXX ANCILLA 3
S 3
H 3
H 3
S 3
NEED A
MZZ A 0 3 1 2
MX A
S ANCILLA
MXX ANCILLA 0 3 1 2
S 3
H 3
H 2
S 2
H 1
NEED A
MZZ A 2 1
MX A
S ANCILLA
MXX ANCILLA 2 1
S 2
H 2
H 1
H 0
S 0
H 3
S 3
MZZ 0 1 2 3
H 0
H 1
MZZ 0 1
MZ 2
H 0
H 3
MZZ 0 3


This is still a naive exporter and the output can be improved. There is a Basis transformation engine for the rotations an ParityMeasurements, which work together with the Local Optimizer. This will clean up some of the Hadamards and S gates.


In [6]:
engines3 = [projectq.cengines.TagRemover(),PermutePi4Front(),MultiqubitMeasurementCliffordEngine(),BasisRotation(), LocalOptimizer()]
eng3 = projectq.MainEngine(backend=OpenSurgeryExporter(output="output_better_instructions.txt"), engine_list=engines3, verbose=True)

qubit1 = eng3.allocate_qubit()
qubit2 = eng3.allocate_qubit()
qubit3 = eng3.allocate_qubit()
qubit4 = eng3.allocate_qubit()

T | qubit1
CNOT | (qubit3, qubit2)
Rx(-cmath.pi/2) | qubit4

CNOT | (qubit2, qubit1)
Rx(cmath.pi/2) | qubit3
Rz(cmath.pi/4) | qubit4

CNOT | (qubit4, qubit1)

Rz(cmath.pi/4) | qubit1
Rz(cmath.pi/2) | qubit2
Rz(cmath.pi/4) | qubit3
Rz(cmath.pi/2) | qubit4

Rx(-cmath.pi/2) | qubit1
Rx(cmath.pi/2) | qubit2
Rx(cmath.pi/2) | qubit3
Rx(cmath.pi/2) | qubit4

Measure | qubit1
Measure | qubit2
Measure | qubit3
Measure | qubit4

eng3.flush()

In [7]:
# the output file now looks like:
with open("output_instructions.txt") as fin:
    for line in fin:
        line = line.strip()
        print(line)

INIT 4
NEED A
MZZ A 0
MX A
S ANCILLA
MXX ANCILLA 0
H 3
S 3
NEED A
MZZ A 3
MX A
S ANCILLA
MXX ANCILLA 3
S 3
H 3
H 3
S 3
NEED A
MZZ A 0 3 1 2
MX A
S ANCILLA
MXX ANCILLA 0 3 1 2
S 3
H 3
H 2
S 2
H 1
NEED A
MZZ A 2 1
MX A
S ANCILLA
MXX ANCILLA 2 1
S 2
H 2
H 1
H 0
S 0
H 3
S 3
MZZ 0 1 2 3
H 0
H 1
MZZ 0 1
MZ 2
H 0
H 3
MZZ 0 3


Since The engine lists tend to get large we added convenience functions that generate an engine-list useful for the compilation to Lattice Surgery. In this engine-list arbitrary gates are also supported as long as there are decompositon rules written to map to Clifford+T


# On to more general circuits...
The provided engine_list is a whole stack of compile engines that should be able to translate an arbitrary circuit into the format needed by arxiv:1808.02892 It first decomposes the arbitrary circuit into Clifford+T and then performs the circuit manipulations that were used before.

In [8]:
from projqext.projectq.setups.surface_codes import lattice_surgery
# this first performs a decomposition in clifford + T and then uses the decomposition above
# engines4 = lattice_surgery.get_engine_list() # first transforms any circuit to clifford+T then to multi-qubit rotations
engines4 = lattice_surgery.SimpleExporterEngineList() # This is the same as before but only using multi-qubit Z rotations
print(engines4)

[<projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fe5dd5d9518>, <projectq.cengines._tagremover.TagRemover object at 0x7fe5dd6062e8>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7fe5dd606320>, <projectq.cengines._optimize.LocalOptimizer object at 0x7fe5dd606278>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fe5dd6063c8>, <projectq.cengines._tagremover.TagRemover object at 0x7fe5dd606400>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7fe5dd606470>, <projectq.cengines._optimize.LocalOptimizer object at 0x7fe5dd606438>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7fe5dd6064a8>, <projectq.cengines._tagremover.TagRemover object at 0x7fe5dd6064e0>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7fe5dd606550>, <projectq.cengines._optimize.LocalOptimizer object at 0x7fe5dd606518>, <projqext.projectq.cengines._permutation_engine._permute_pi4_front.PermutePi4Front object 

In [9]:
eng4 = projectq.MainEngine(backend=OpenSurgeryExporter("output_final_example.txt"), engine_list=engines4, verbose=True)


# now add a general circuit any gate can be used as long as Decomposition rules to decompose it into Clifford+T exist

# TODO: need decomposition of rotation gates into Clifford + T